DerivaML is a class library built on the Deriva Scientific Asset management system that is designed to help simplify a number of the basic operations associated with building and testing ML libraries based on common toolkits such as TensorFlow.  This notebook reviews the basic features of the DerivaML library.

In [ ]:
from fontTools.subset import subset_features
%load_ext autoreload
%autoreload 2

In [ ]:
import builtins
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
from deriva_ml.deriva_ml_base import ColumnDefinition, BuiltinTypes
from deriva_ml.schema_setup.test_catalog import create_test_catalog, DemoML
from deriva_ml.execution_configuration import ExecutionConfiguration, Workflow, Execution, WorkflowTerm, Term
import itertools
from pathlib import Path
import pandas as pd
import tempfile
import random

Set the details for the catalog we want and authenticate to the server if needed.

In [ ]:
hostname = 'dev.eye-ai.org'
domain_schema = 'demo-schema'

gnl = GlobusNativeLogin(host=hostname)
if gnl.is_logged_in([hostname]):
    print("You are already logged in.")
else:
    gnl.login([hostname], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")


Create a test catalog and get an instance of the DerivaML class.

In [ ]:
test_catalog = create_test_catalog(hostname, domain_schema)
ml_instance = DemoML(hostname, test_catalog.catalog_id)

## Create New Features

A feature is a set of values that are attached to a table in the DerivaML catalog. Instances of features are distingushed from one another by the ID of the execution that produced the feature value. The execution could be the result of a program, or it could be a manual process by which a person defines a set of values

To create a new feature, we need to know the name of the feature, the table to which it is attached, and the set of values that make up the feature.  The values could be terms from a controlled vocabulary, a set of one or more file based assets, or other values, such as integers, or strings. However, use of strings outside of controlled vocabularies is discouraged.

In [ ]:
# Prerequests for our feature, which will include a CV term and asset.

# Create a vocabulary and add a term to it to use in our features.
ml_instance.create_vocabulary("SubjectStatus", comment="A vocab for image quality")
ml_instance.add_term("SubjectStatus", "Sick", description="A sick subject")
ml_instance.add_term("SubjectStatus", "Well", description="A healthy subject")

box_asset = ml_instance.create_asset("BoundingBox", comment="A asset")

# Now lets create and upload a simple asset.
with tempfile.TemporaryDirectory() as tmpdirname:
    assetdir = Path(tmpdirname) / "BoundingBox"
    assetdir.mkdir(parents=True, exist_ok=True)
    for i in range(10):
        with builtins.open(assetdir / f"box{i}.txt", "w") as fp:
            fp.write(f"Hi there from box {i}")
    ml_instance.upload_assets(assetdir)

In [ ]:
# We are going to have three values in our feature, a controlled vocabulary term from the vocabulary FeatureValue, a file asset and 
# an integer value which we will call "TestCol"
ml_instance.create_feature("SubjectStatus", "Subject",
                                        terms=["SubjectStatus"],
                                        metadata=[ColumnDefinition(name='Scale', type=BuiltinTypes.int2)])

ml_instance.create_feature("BoundingBox", "Image",
                                        assets=[box_asset],
                                        metadata=[ColumnDefinition(name='Scale', type=BuiltinTypes.int2)])


print([f.feature_name for f in ml_instance.find_features("Subject")])
print([f.feature_name for f in ml_instance.find_features("Image")])


Now we can add some features to our images.  To streamline the creation of new feature, we create a class that is specific to the arguments required to create it.

In [ ]:
ImageFeatureClass = ml_instance.feature_record_class("Image", "BoundingBox")
SubjectFeatureClass = ml_instance.feature_record_class("Subject", "SubjectStatus")

print("Image Feature:")
display(ImageFeatureClass.model_fields)
print("Subject Feature:")
display(SubjectFeatureClass.model_fields)


Now using TestFeatureClass, we can create some instances of the feature and add it.  We must have a exeuction_rid in order to define the feature.

In [ ]:
config = ExecutionConfiguration(
    execution=Execution(description="Sample Execution"), 
    workflow=Workflow(
        name="Sample Workflow", 
        url="https://github.com/informatics-isi-edu/deriva-ml/blob/main/pyproject.toml", 
        workflow_type="Sample Workflow"), 
    workflow_terms=[WorkflowTerm(term=Term.workflow, name="Sample Workflow", description="Sample Workflow Example")],
    description="Our Sample Workflow instance")
configuration_record = ml_instance.initialize_execution(config)
execution_rid = configuration_record.execution_rid

In [ ]:
# Get some images to attach the feature value to.
image_rids = [i['RID'] for i in ml_instance.domain_path.tables['Image'].entities().fetch()]
asset_rids = [i['RID'] for i in ml_instance.domain_path.tables['BoundingBox'].entities().fetch()]
subject_rids = [i['RID'] for i in ml_instance.domain_path.tables['Subject'].entities().fetch()]

# Now create a list of features using the feature creation class returned by create_feature.
image_feature_list = [ImageFeatureClass(
    Image=image_rid,
    Execution=execution_rid,
    BoundingBox=asset_rid,
    Scale=random.randint(1, 10)) for image_rid, asset_rid in zip(image_rids, itertools.cycle(asset_rids))]

subject_feature_list = [SubjectFeatureClass(
    Subject=subject_rid,
    Execution=execution_rid,
    SubjectStatus=status,
    Scale=random.randint(1, 10)) for subject_rid, status in zip(subject_rids, itertools.cycle(["Sick", "Well"]))]

ml_instance.add_features(image_feature_list)
ml_instance.add_features(subject_feature_list)

In [ ]:
system_columns = ['RCT', 'RMT', 'RCB', 'RMB']

display(pd.DataFrame(ml_instance.list_feature("Subject", "SubjectStatus")).drop(columns=system_columns))
display(pd.DataFrame(ml_instance.list_feature("Image", "BoundingBox")).drop(columns=system_columns))


## Computing Features

In the previous example, we used the API to create new features. However, creating new feature values is also integrated into the process fo managing an execution in deriva-ml.

First we will create a new execution for our computation.

In [ ]:
config = ExecutionConfiguration(
    execution=Execution(description="Our second execution"),
    workflow=Workflow(
        name="Sample Workflow",
        url="https://github.com/informatics-isi-edu/deriva-ml/blob/main/pyproject.toml",
        workflow_type="Sample Workflow"),
    workflow_terms=[WorkflowTerm(term=Term.workflow, name="Sample Workflow", description="Sample Workflow Example")],
    description="Our Sample Workflow instance")
configuration_record = ml_instance.initialize_execution(config)
execution_rid = configuration_record.execution_rid

In [ ]:
ml_instance.add_dataset_element_type("Subject")
ml_instance.add_term("Dataset_Type", "Subject Dataset", description="Subject Dataset")
subject_dataset = ml_instance.create_dataset("Subject Dataset", description="Subject Dataset")
ml_instance.add_dataset_members(subject_dataset, subject_rids)

In [ ]:
bag_path = ml_instance.materialize_dataset_bag(subject_dataset)[0]

In [ ]:
bag_path

In [ ]:
test_catalog.delete_ermrest_catalog(really=True)